# Tickers Module

This notebook demonstrates the ticker utilities from `specparser.amt.tickers`.

## What is a Ticker Schema?

A **ticker schema** is a template that defines all the data sources (tickers) needed for an asset:
- **Market tickers**: Price data (e.g., `AAPL US Equity` with field `PX_LAST`)
- **Vol tickers**: Implied volatility data (Near and Far month)
- **Hedge tickers**: Hedging instrument data

## Ticker Sources

| Source | Description |
|--------|-------------|
| `BBG` | Bloomberg direct ticker |
| `BBGfc` | Bloomberg futures spec (needs expansion) |
| `CV` | CitiVelocity (for swaption vols) |
| `calc` | Computed tickers (for swap hedges) |
| `nBBG` | Normalized Bloomberg (not in chain) |

## Split Ticker Format

When tickers change over time (e.g., LIBOR → SOFR transition):
```
ticker1:YYYY-MM:ticker2
```
- Before the date: use `ticker1`
- After the date: use `ticker2`

In [24]:
# Setup: Imports and helper function
import pandas as pd
from specparser.amt import (
    # Ticker schemas
    get_tschemas,
    find_tschemas,
    # Futures expansion
    fut_spec2ticker,
    fut_norm2act,
    fut_act2norm,
    clear_normalized_cache,
    # Split tickers
    _split_ticker,
    # Period tickers
    find_tickers,
    find_tickers_ym,
    # Straddle tickers and prices
    filter_tickers,
    get_prices,
    actions,
    get_straddle_actions,
    get_straddle_valuation,
    load_all_prices,
    # Table utilities
    table_to_rows,
    table_column,
    print_table,
)

# Data paths (relative to this notebook)
AMT_PATH = "../data/amt.yml"
PRICES_PATH = "../data/prices.parquet"
CHAIN_PATH = None # "../data/futs.csv"
OVERRIDE_PATH = "../data/overrides.csv"

def show_table(tbl):
    """Display a table as a pandas DataFrame for pretty rendering."""
    if tbl.get("orientation") in ("column", "arrow"):
        t = table_to_rows(tbl)
    else:
        t = tbl
    return pd.DataFrame(t["rows"], columns=t["columns"])

---
## 1. Ticker Schemas

### `get_tschemas(path, underlying)`

Get all tickers for a single asset. Returns a table with columns:
- `asset`: The underlying asset
- `cls`: Asset class (Commodity, Equity, Currency, etc.)
- `type`: Ticker type (Market, Vol, Hedge)
- `param`: Parameter name (e.g., Near, Far, hedge, hedge1)
- `source`: Data source (BBG, BBGfc, CV, calc)
- `ticker`: The ticker symbol or spec
- `field`: The data field to fetch

In [25]:
# Get tickers for a commodity asset
tickers = get_tschemas(AMT_PATH, "CL Comdty")
show_table(tickers)

,asset,cls,type,param,source,ticker,field
0,CL Comdty,Commodity,Market,-,BBG,CL1 Comdty,PX_LAST
1,CL Comdty,Commodity,Vol,Near,BBG,CL1 Comdty,1ST_MTH_IMPVOL_100.0%MNY_DF
2,CL Comdty,Commodity,Vol,Far,BBG,CL1 Comdty,2ND_MTH_IMPVOL_100.0%MNY_DF
3,CL Comdty,Commodity,Hedge,fut,BBGfc,"generic:CL1 Comdty,fut_code:CL,fut_month_map:G...",PX_LAST


In [3]:
# Get tickers for an equity asset
tickers = get_tschemas(AMT_PATH, "SPX Index")
show_table(tickers)

,asset,cls,type,param,source,ticker,field
0,SPX Index,Equity,Market,-,BBG,SPX Index,PX_LAST
1,SPX Index,Equity,Vol,Near,BBG,SPX Index,30DAY_IMPVOL_100.0%MNY_DF
2,SPX Index,Equity,Vol,Far,BBG,SPX Index,60DAY_IMPVOL_100.0%MNY_DF
3,SPX Index,Equity,Hedge,hedge,BBG,SPX Index,PX_LAST


In [4]:
# Get tickers for a currency asset
tickers = get_tschemas(AMT_PATH, "EURUSD Curncy")
show_table(tickers)

,asset,cls,type,param,source,ticker,field
0,EURUSD Curncy,Currency,Market,-,BBG,EURUSD Curncy,PX_LAST
1,EURUSD Curncy,Currency,Vol,Near,BBG,EURUSDV1M CMPN Curncy,PX_LAST
2,EURUSD Curncy,Currency,Hedge,hedge,BBG,EURUSD Curncy,PX_LAST


In [10]:
# Get tickers for a currency asset
tickers = get_tschemas(AMT_PATH, "IBOXUMAE Curncy")
show_table(tickers)

,asset,cls,type,param,source,ticker,field
0,IBOXUMAE Curncy,Rate,Market,-,BBG,IBOXUMAE CBBT Curncy,PX_LAST
1,IBOXUMAE Curncy,Rate,Market,-,BBG,USSWAP5 CMPN Curncy,PX_LAST
2,IBOXUMAE Curncy,Rate,Vol,Near,CV,,CREDIT.CVOL.CDX_NA_IG.1M.CVOL_ATM.CVOL_VOL.CVO...
3,IBOXUMAE Curncy,Rate,Hedge,hedge,BBG,IBOXUMAE CBBT Curncy,PX_LAST
4,IBOXUMAE Curncy,Rate,Hedge,hedge1,BBG,USSWAP5 CMPN Curncy:2023-06:USOSFR5 Curncy,PX_LAST


### `find_tschemas(path, pattern, live_only)`

Find ticker schemas for multiple assets matching a regex pattern.

In [11]:
# Find tickers for all commodities
commodity_tickers = find_tschemas(AMT_PATH, pattern=".", live_only=True)
print(f"Found {len(commodity_tickers['rows'])} ticker rows for commodities")
show_table(commodity_tickers)

Found 1067 ticker rows for commodities


,asset,cls,type,param,source,ticker,field
0,LA Comdty,Commodity,Market,-,BBG,LA1 Comdty,PX_LAST
1,LA Comdty,Commodity,Vol,Near,BBG,LA1 Comdty,1ST_MTH_IMPVOL_100.0%MNY_DF
2,LA Comdty,Commodity,Vol,Far,BBG,LA1 Comdty,2ND_MTH_IMPVOL_100.0%MNY_DF
3,LA Comdty,Commodity,Hedge,fut,BBGfc,"generic:LA1 Comdty,fut_code:LA,fut_month_map:F...",PX_LAST
4,LP Comdty,Commodity,Market,-,BBG,LP1 Comdty,PX_LAST
...,...,...,...,...,...,...,...
1062,DVN US Equity,SingleStock,Hedge,hedge,BBG,DVN US Equity,PX_LAST
1063,LOW US Equity,SingleStock,Market,-,BBG,LOW US Equity,PX_LAST
1064,LOW US Equity,SingleStock,Vol,Near,BBG,LOW US Equity,30DAY_IMPVOL_100.0%MNY_DF
1065,LOW US Equity,SingleStock,Vol,Far,BBG,LOW US Equity,60DAY_IMPVOL_100.0%MNY_DF


In [6]:
# Find tickers for specific assets (CL, GC, SI)
tickers = find_tschemas(AMT_PATH, pattern="^(CL|GC|SI) ", live_only=True)
print(f"CL, GC, SI tickers: {len(tickers['rows'])} rows")
show_table(tickers)

CL, GC, SI tickers: 12 rows


,asset,cls,type,param,source,ticker,field
0,CL Comdty,Commodity,Market,-,BBG,CL1 Comdty,PX_LAST
1,CL Comdty,Commodity,Vol,Near,BBG,CL1 Comdty,1ST_MTH_IMPVOL_100.0%MNY_DF
2,CL Comdty,Commodity,Vol,Far,BBG,CL1 Comdty,2ND_MTH_IMPVOL_100.0%MNY_DF
3,CL Comdty,Commodity,Hedge,fut,BBGfc,"generic:CL1 Comdty,fut_code:CL,fut_month_map:G...",PX_LAST
4,GC Comdty,Commodity,Market,-,BBG,GC1 Comdty,PX_LAST
5,GC Comdty,Commodity,Vol,Near,BBG,GC1 Comdty,1ST_MTH_IMPVOL_100.0%MNY_DF
6,GC Comdty,Commodity,Vol,Far,BBG,GC1 Comdty,2ND_MTH_IMPVOL_100.0%MNY_DF
7,GC Comdty,Commodity,Hedge,fut,BBGfc,"generic:GC1 Comdty,fut_code:GC,fut_month_map:G...",PX_LAST
8,SI Comdty,Commodity,Market,-,BBG,SI1 Comdty,PX_LAST
9,SI Comdty,Commodity,Vol,Near,BBG,SI1 Comdty,1ST_MTH_IMPVOL_100.0%MNY_DF


---
## 2. Understanding Ticker Types

### Market Tickers
- **Source**: Always `BBG`
- **Type**: `Market`
- **Purpose**: Get market price data
- **Field**: Usually `PX_LAST`

### Vol Tickers
- **Sources**: `BBG`, `CV` (CitiVelocity), `BBG_LMEVOL`
- **Type**: `Vol`
- **Params**: `Near` (near-month vol), `Far` (far-month vol)
- **Purpose**: Get implied volatility for option pricing

### Hedge Tickers
- **Sources**: `BBG` (nonfut), `BBGfc` (futures), `calc` (computed), `cds`
- **Type**: `Hedge`
- **Params**: `hedge`, `hedge1`, `hedge2`, `hedge3`, `fut`
- **Purpose**: Get prices for hedging instruments

In [12]:
# Analyze ticker types for an asset
tickers = get_tschemas(AMT_PATH, "CL Comdty")

print("Ticker breakdown for CL Comdty:")
print()
for row in tickers["rows"]:
    asset, cls, typ, param, source, ticker, field = row
    print(f"Type: {typ:8s} | Param: {param:8s} | Source: {source:6s}")
    print(f"  Ticker: {ticker}")
    print(f"  Field:  {field}")
    print()

Ticker breakdown for CL Comdty:

Type: Market   | Param: -        | Source: BBG   
  Ticker: CL1 Comdty
  Field:  PX_LAST

Type: Vol      | Param: Near     | Source: BBG   
  Ticker: CL1 Comdty
  Field:  1ST_MTH_IMPVOL_100.0%MNY_DF

Type: Vol      | Param: Far      | Source: BBG   
  Ticker: CL1 Comdty
  Field:  2ND_MTH_IMPVOL_100.0%MNY_DF

Type: Hedge    | Param: fut      | Source: BBGfc 
  Ticker: generic:CL1 Comdty,fut_code:CL,fut_month_map:GHJKMNQUVXZF,min_year_offset:0,market_code:Comdty
  Field:  PX_LAST



In [13]:
# Count ticker types across all live assets
from collections import Counter

all_tickers = find_tschemas(AMT_PATH, pattern=".", live_only=True)

types = Counter(row[2] for row in all_tickers["rows"])  # type is column 2
sources = Counter(row[4] for row in all_tickers["rows"])  # source is column 4

print("Ticker types:")
for t, count in types.most_common():
    print(f"  {t}: {count}")

print("\nTicker sources:")
for s, count in sources.most_common():
    print(f"  {s}: {count}")

Ticker types:
  Market: 534
  Vol: 291
  Hedge: 242

Ticker sources:
  BBG: 961
  calc: 64
  BBGfc: 21
  CV: 21


---
## 3. Futures Ticker Expansion

### `fut_spec2ticker(spec, year, month)`

Futures tickers use a spec string that needs to be expanded to actual contract tickers.

**Spec format:**
```
fut_code:CL,fut_month_map:GHJKMNQUVXZF,min_year_offset:0,market_code:Comdty
```

**Month map:** 12 characters mapping Jan-Dec to contract codes:
- Standard: `FGHJKMNQUVXZ` (F=Jan, G=Feb, H=Mar, J=Apr, K=May, M=Jun, N=Jul, Q=Aug, U=Sep, V=Oct, X=Nov, Z=Dec)

In [14]:
# Example: Expand a crude oil futures spec
spec = "fut_code:CL,fut_month_map:GHJKMNQUVXZF,min_year_offset:0,market_code:Comdty"

# Get ticker for June 2024
ticker = fut_spec2ticker(spec, 2024, 6)
print(f"June 2024: {ticker}")

# Get ticker for December 2024
ticker = fut_spec2ticker(spec, 2024, 12)
print(f"December 2024: {ticker}")

June 2024: CLN2024 Comdty
December 2024: CLF2025 Comdty


In [15]:
# Generate a full year of futures tickers
spec = "fut_code:CL,fut_month_map:GHJKMNQUVXZF,min_year_offset:0,market_code:Comdty"

print("CL Comdty futures calendar for 2024:")
print("-" * 40)
months = ["Jan", "Feb", "Mar", "Apr", "May", "Jun", "Jul", "Aug", "Sep", "Oct", "Nov", "Dec"]
for month_num, month_name in enumerate(months, 1):
    ticker = fut_spec2ticker(spec, 2024, month_num)
    print(f"{month_name:3s}: {ticker}")

CL Comdty futures calendar for 2024:
----------------------------------------
Jan: CLG2024 Comdty
Feb: CLH2024 Comdty
Mar: CLJ2024 Comdty
Apr: CLK2024 Comdty
May: CLM2024 Comdty
Jun: CLN2024 Comdty
Jul: CLQ2024 Comdty
Aug: CLU2024 Comdty
Sep: CLV2024 Comdty
Oct: CLX2024 Comdty
Nov: CLZ2024 Comdty
Dec: CLF2025 Comdty


### Understanding Month Maps

Different assets have different delivery months. The month map encodes this:

| Position | Standard | Meaning |
|----------|----------|--------|
| 1 (Jan)  | F        | January contract |
| 2 (Feb)  | G        | February contract |
| 3 (Mar)  | H        | March contract |
| ...      | ...      | ... |
| 12 (Dec) | Z        | December contract |

Some assets only trade certain months (e.g., quarterly). In these cases, the month map repeats characters to indicate which contract to use.

In [16]:
# Example: An asset with quarterly contracts only (H, M, U, Z)
# The month map might look like: HHHMMMUUUZZZ
# - Jan, Feb, Mar all use H (March)
# - Apr, May, Jun all use M (June)
# - Jul, Aug, Sep all use U (September)
# - Oct, Nov, Dec all use Z (December)

quarterly_spec = "fut_code:ED,fut_month_map:HHHMMMUUUZZZ,min_year_offset:0,market_code:Comdty"

print("Quarterly contract example:")
for month_num, month_name in enumerate(months, 1):
    ticker = fut_spec2ticker(quarterly_spec, 2024, month_num)
    print(f"{month_name:3s}: {ticker}")

Quarterly contract example:
Jan: EDH2024 Comdty
Feb: EDH2024 Comdty
Mar: EDH2024 Comdty
Apr: EDM2024 Comdty
May: EDM2024 Comdty
Jun: EDM2024 Comdty
Jul: EDU2024 Comdty
Aug: EDU2024 Comdty
Sep: EDU2024 Comdty
Oct: EDZ2024 Comdty
Nov: EDZ2024 Comdty
Dec: EDZ2024 Comdty


---
## 4. Normalized vs Actual Tickers

Bloomberg uses specific ticker formats that may differ from the "normalized" format we generate.

**Example:**
- Normalized: `CLN2024 Comdty`
- Actual BBG: `CL N24 Comdty`

The `fut_norm2act()` and `fut_act2norm()` functions convert between these formats using a CSV lookup table.

In [19]:
# Check if chain CSV exists and demonstrate conversion
import os

if os.path.exists(CHAIN_PATH if CHAIN_PATH is not None else ""):
    print(f"Chain CSV found: {CHAIN_PATH}")
    
    # Example conversion
    normalized = "CLN2024 Comdty"
    actual = fut_norm2act(CHAIN_PATH, normalized)
    print(f"Normalized: {normalized}")
    print(f"Actual:     {actual}")
    
    # Reverse conversion
    if actual:
        back = fut_act2norm(CHAIN_PATH, actual)
        print(f"Back:       {back}")
else:
    print(f"Chain CSV not found: {CHAIN_PATH}")
    print("Skipping normalized/actual conversion examples.")

Chain CSV not found: None
Skipping normalized/actual conversion examples.


In [20]:
# Clear the lookup cache if needed
clear_normalized_cache()
print("Normalized/actual cache cleared")

Normalized/actual cache cleared


---
## 5. Split Tickers (Ticker Transitions)

When Bloomberg changes ticker symbols, we use "split tickers" to handle the transition:

```
ticker1:YYYY-MM:ticker2
```

The `_split_ticker()` function expands this into two rows with date constraints:
- `param<YYYY-MM` - use ticker1 before the date
- `param>YYYY-MM` - use ticker2 after the date

In [21]:
# Example: LIBOR to SOFR transition
split_ticker = "USSWAP5 CMPN Curncy:2023-06:USOSFR5 Curncy"
param = "hedge1"

result = _split_ticker(split_ticker, param)

print(f"Split ticker: {split_ticker}")
print(f"Original param: {param}")
print()
print("Expanded to:")
for ticker, new_param in result:
    print(f"  Ticker: {ticker}")
    print(f"  Param:  {new_param}")
    print()

Split ticker: USSWAP5 CMPN Curncy:2023-06:USOSFR5 Curncy
Original param: hedge1

Expanded to:
  Ticker: USSWAP5 CMPN Curncy
  Param:  hedge1<2023-06

  Ticker: USOSFR5 Curncy
  Param:  hedge1>2023-06



In [ ]:
# A non-split ticker returns unchanged
regular_ticker = "SPY US Equity"
result = _split_ticker(regular_ticker, "hedge")

print(f"Regular ticker: {regular_ticker}")
print(f"Result: {result}")
print("(Single element list - no split)")

---
## 6. Tickers for Specific Periods

### `find_tickers_ym(path, pattern, live_only, year, month, chain_csv)`

Get tickers for a specific year/month. This:
1. Expands BBGfc specs to actual futures tickers
2. Applies date constraints from split tickers
3. Optionally converts normalized to actual tickers via chain CSV

In [22]:
# Get tickers for CL Comdty in June 2024
tickers = find_tickers_ym(AMT_PATH, "^CL Comdty$", True, 2024, 6, CHAIN_PATH)
print(f"CL Comdty tickers for June 2024: {len(tickers['rows'])} rows")
show_table(tickers)

CL Comdty tickers for June 2024: 4 rows


,asset,cls,type,param,source,ticker,field
0,CL Comdty,Commodity,Market,-,BBG,CL1 Comdty,PX_LAST
1,CL Comdty,Commodity,Vol,Near,BBG,CL1 Comdty,1ST_MTH_IMPVOL_100.0%MNY_DF
2,CL Comdty,Commodity,Vol,Far,BBG,CL1 Comdty,2ND_MTH_IMPVOL_100.0%MNY_DF
3,CL Comdty,Commodity,Hedge,hedge,BBG,CLN2024 Comdty,PX_LAST


### `find_tickers(path, pattern, live_only, start_year, end_year, chain_csv)`

Get unique tickers across a year range. Useful for knowing all tickers needed for a backtest period.

In [23]:
# Get all unique tickers for commodities in 2024
tickers = find_tickers(
    AMT_PATH, 
    pattern="Comdty$", 
    live_only=True,
    start_year=2024, 
    end_year=2024,
    chain_csv=CHAIN_PATH
)
print(f"Unique commodity tickers for 2024: {len(tickers['rows'])} rows")
show_table(tickers)

Unique commodity tickers for 2024: 194 rows


,asset,cls,type,param,source,ticker,field
0,LA Comdty,Commodity,Market,-,BBG,LA1 Comdty,PX_LAST
1,LA Comdty,Commodity,Vol,Near,BBG,LA1 Comdty,1ST_MTH_IMPVOL_100.0%MNY_DF
2,LA Comdty,Commodity,Vol,Far,BBG,LA1 Comdty,2ND_MTH_IMPVOL_100.0%MNY_DF
3,LA Comdty,Commodity,Hedge,hedge,BBG,LAF2024 Comdty,PX_LAST
4,LP Comdty,Commodity,Market,-,BBG,LP1 Comdty,PX_LAST
...,...,...,...,...,...,...,...
189,RX Comdty,Rate,Hedge,hedge,BBG,RXH2025 Comdty,PX_LAST
190,TY Comdty,Rate,Hedge,hedge,BBG,TYH2025 Comdty,PX_LAST
191,US Comdty,Rate,Hedge,hedge,BBG,USH2025 Comdty,PX_LAST
192,FV Comdty,Rate,Hedge,hedge,BBG,FVH2025 Comdty,PX_LAST


---
## 7. Straddle Tickers

### `filter_tickers(asset, year, month, i, amt_path, chain_path)`

Get tickers for a specific straddle component with filtering:

**Filtering Rules:**
- **Market**: Always excluded (not needed for straddle valuation)
- **Vol/Near**: Kept only if straddle entry code (`ntrc`) is `"N"`, param → `"vol"`
- **Vol/Far**: Kept only if straddle entry code (`ntrc`) is `"F"`, param → `"vol"`
- **Hedge**: Always kept

**Parameters:**
- `i`: Straddle component index (0, 1, 2, ... for multiple schedule components)

In [12]:
# Get straddle tickers for CL Comdty, June 2024, first component
tickers = filter_tickers(
    asset="CL Comdty",
    year=2024,
    month=6,
    i=0,  # First straddle component
    amt_path=AMT_PATH,
    chain_path=CHAIN_PATH
)
print("Straddle tickers for CL Comdty, June 2024, component 0:")
show_table(tickers)

Straddle tickers for CL Comdty, June 2024, component 0:


,asset,straddle,param,source,ticker,field
0,CL Comdty,|2024-05|2024-06|N|0|OVERRIDE||33.3|,vol,BBG,CL1 Comdty,1ST_MTH_IMPVOL_100.0%MNY_DF
1,CL Comdty,|2024-05|2024-06|N|0|OVERRIDE||33.3|,hedge,BBG,CLN24 Comdty,PX_LAST


In [7]:
# Compare different straddle components (i=0 vs i=1)
for i in range(2):
    tickers = filter_tickers(
        asset="CL Comdty",
        year=2024,
        month=6,
        i=i,
        amt_path=AMT_PATH,
        chain_path=CHAIN_PATH
    )
    print(f"Component {i}: {len(tickers['rows'])} ticker rows")
    # Show the straddle string
    if tickers['rows']:
        straddle = tickers['rows'][0][1]  # straddle is column 1
        print(f"  Straddle: {straddle}")
    print()

Component 0: 2 ticker rows
  Straddle: |2024-05|2024-06|N|0|OVERRIDE||33.3|

Component 1: 2 ticker rows
  Straddle: |2024-05|2024-06|N|5|OVERRIDE||33.3|



---
## 8. Getting Prices for Straddles

The tickers module provides a pipeline for getting daily prices with action/strike columns:

1. **`get_prices()`** - Get daily prices for a straddle (dates × tickers → wide table)
2. **`actions()`** - Add action, model, and strike columns to a prices table
3. **`get_straddle_actions()`** - Convenience function composing `get_prices()` + `actions()`
4. **`get_straddle_valuation()`** - Add valuation columns (mv, delta, pnl)

In [8]:
# Load prices into memory for fast lookups
load_all_prices(PRICES_PATH)
print("Prices loaded")

Prices loaded


In [9]:
# Get daily prices for a straddle
prices_table = get_prices(
    "CL Comdty",
    2024,
    6,
    0,  # First straddle component
    AMT_PATH,
    CHAIN_PATH,
    PRICES_PATH
)
print(f"Prices table: {len(prices_table['rows'])} rows")
print(f"Columns: {prices_table['columns']}")
show_table(prices_table)

Prices table: 61 rows
Columns: ['asset', 'straddle', 'date', 'vol', 'hedge']


,asset,straddle,date,vol,hedge
0,CL Comdty,|2024-05|2024-06|N|0|OVERRIDE||33.3|,2024-05-01,26.36,78.44
1,CL Comdty,|2024-05|2024-06|N|0|OVERRIDE||33.3|,2024-05-02,25.11,78.48
2,CL Comdty,|2024-05|2024-06|N|0|OVERRIDE||33.3|,2024-05-03,24.85,77.76
3,CL Comdty,|2024-05|2024-06|N|0|OVERRIDE||33.3|,2024-05-04,none,none
4,CL Comdty,|2024-05|2024-06|N|0|OVERRIDE||33.3|,2024-05-05,none,none
...,...,...,...,...,...
56,CL Comdty,|2024-05|2024-06|N|0|OVERRIDE||33.3|,2024-06-26,23.61,none
57,CL Comdty,|2024-05|2024-06|N|0|OVERRIDE||33.3|,2024-06-27,23.97,none
58,CL Comdty,|2024-05|2024-06|N|0|OVERRIDE||33.3|,2024-06-28,24.45,none
59,CL Comdty,|2024-05|2024-06|N|0|OVERRIDE||33.3|,2024-06-29,none,none


In [ ]:
# Add action, model, and strike columns
actions_table = actions(prices_table, AMT_PATH, OVERRIDE_PATH)
print(f"Actions table columns: {actions_table['columns']}")
show_table(actions_table)

In [ ]:
# Convenience function: get_prices + actions in one call
full_table = get_straddle_actions(
    "CL Comdty",
    2024,
    6,
    0,
    AMT_PATH,
    CHAIN_PATH,
    PRICES_PATH,
    OVERRIDE_PATH
)
print(f"Full straddle actions: {len(full_table['rows'])} rows")
show_table(full_table)

In [ ]:
# Get straddle valuation with PnL columns
valuation_table = get_straddle_valuation(
    "CL Comdty",
    2024,
    6,
    0,
    AMT_PATH,
    CHAIN_PATH,
    PRICES_PATH,
    OVERRIDE_PATH
)
print(f"Valuation columns: {valuation_table['columns']}")
show_table(valuation_table)

### Example: Different Asset Types

In [ ]:
# Currency straddle
currency_actions = get_straddle_actions(
    "EURUSD Curncy",
    2024,
    1,
    0,
    AMT_PATH,
    CHAIN_PATH,
    PRICES_PATH,
    OVERRIDE_PATH
)
print(f"EURUSD Curncy: {len(currency_actions['rows'])} rows")
show_table(currency_actions)

In [6]:
# Metal commodity (LA = Aluminum)
metal_actions = get_straddle_actions(
    "LA Comdty",
    2024,
    1,
    0,
    AMT_PATH,
    CHAIN_PATH,
    PRICES_PATH,
    OVERRIDE_PATH
)
print(f"LA Comdty: {len(metal_actions['rows'])} rows")
show_table(metal_actions)

LA Comdty: 62 rows


,asset,straddle,date,vol,hedge,action,model,strike_vol,strike,expiry
0,LA Comdty,|2023-12|2024-01|N|0|OVERRIDE||33.3|,2023-12-01,13.38,2196.5,-,ES,-,-,-
1,LA Comdty,|2023-12|2024-01|N|0|OVERRIDE||33.3|,2023-12-02,none,none,-,ES,-,-,-
2,LA Comdty,|2023-12|2024-01|N|0|OVERRIDE||33.3|,2023-12-03,none,none,-,ES,-,-,-
3,LA Comdty,|2023-12|2024-01|N|0|OVERRIDE||33.3|,2023-12-04,15.36,2166.25,-,ES,-,-,-
4,LA Comdty,|2023-12|2024-01|N|0|OVERRIDE||33.3|,2023-12-05,15.22,2145.5,-,ES,-,-,-
...,...,...,...,...,...,...,...,...,...,...
57,LA Comdty,|2023-12|2024-01|N|0|OVERRIDE||33.3|,2024-01-27,none,none,-,ES,-,-,-
58,LA Comdty,|2023-12|2024-01|N|0|OVERRIDE||33.3|,2024-01-28,none,none,-,ES,-,-,-
59,LA Comdty,|2023-12|2024-01|N|0|OVERRIDE||33.3|,2024-01-29,18.68,none,-,ES,-,-,-
60,LA Comdty,|2023-12|2024-01|N|0|OVERRIDE||33.3|,2024-01-30,none,none,-,ES,-,-,-


---
## 9. Practical Examples

### Example 1: Analyze Ticker Sources by Asset Class

In [ ]:
# Count ticker sources by asset class pattern
from collections import Counter

patterns = [
    ("Comdty$", "Commodities"),
    ("Index$", "Indices"),
    ("Curncy$", "Currencies"),
    ("Equity$", "Equities"),
]

print("Ticker sources by asset class:")
print("-" * 50)

for pattern, name in patterns:
    tickers = find_tschemas(AMT_PATH, pattern, live_only=True)
    if tickers["rows"]:
        sources = Counter(row[4] for row in tickers["rows"])
        print(f"\n{name}:")
        for source, count in sources.most_common():
            print(f"  {source}: {count}")

### Example 2: Find Assets with Split Tickers

In [ ]:
# Find assets that have ticker transitions
import re

all_tickers = find_tschemas(AMT_PATH, ".", live_only=True)

split_assets = set()
for row in all_tickers["rows"]:
    ticker = row[5]  # ticker column
    # Check for split ticker format: YYYY-MM pattern between colons
    if re.search(r':\d{4}-\d{2}:', ticker):
        split_assets.add(row[0])

if split_assets:
    print(f"Assets with split tickers ({len(split_assets)}):")
    for asset in sorted(split_assets):
        print(f"  {asset}")
else:
    print("No assets with split tickers found")

### Example 3: Build a Ticker Inventory

In [ ]:
# Build a summary of all unique tickers needed
all_tickers = find_tschemas(AMT_PATH, ".", live_only=True)

# Group by source
by_source = {}
for row in all_tickers["rows"]:
    source = row[4]
    ticker = row[5]
    if source not in by_source:
        by_source[source] = set()
    by_source[source].add(ticker)

print("Ticker inventory summary:")
print("-" * 40)
for source in sorted(by_source.keys()):
    tickers = by_source[source]
    print(f"\n{source}: {len(tickers)} unique tickers")
    # Show first 3 examples
    for t in list(tickers)[:3]:
        print(f"  - {t}")
    if len(tickers) > 3:
        print(f"  ... and {len(tickers) - 3} more")

---
## 10. Summary

### Ticker Schema Functions

| Function | Description |
|----------|-------------|
| `get_tschemas(path, underlying)` | Get ticker schemas for one asset |
| `find_tschemas(path, pattern, live_only)` | Find ticker schemas by regex pattern |

### Futures Expansion Functions

| Function | Description |
|----------|-------------|
| `fut_spec2ticker(spec, year, month)` | Convert BBGfc spec to ticker |
| `fut_norm2act(csv, ticker)` | Normalized → actual ticker |
| `fut_act2norm(csv, ticker)` | Actual → normalized ticker |
| `clear_normalized_cache()` | Clear lookup cache |

### Split Ticker Functions

| Function | Description |
|----------|-------------|
| `_split_ticker(ticker, param)` | Expand split ticker format |

### Period-Specific Functions

| Function | Description |
|----------|-------------|
| `find_tickers_ym(...)` | Get tickers for one month |
| `find_tickers(...)` | Get unique tickers for year range |
| `filter_tickers(...)` | Get filtered tickers for a straddle |

### Price & Action Functions

| Function | Description |
|----------|-------------|
| `load_all_prices(parquet)` | Load prices into memory for fast lookup |
| `get_prices(...)` | Get daily prices for a straddle |
| `actions(prices_table, ...)` | Add action/strike columns |
| `get_straddle_actions(...)` | Convenience: get_prices + actions |
| `get_straddle_valuation(...)` | Add valuation (mv, delta, pnl) columns |

### Schema Columns

| Column | Description |
|--------|-------------|
| `asset` | Asset underlying |
| `cls` | Asset class |
| `type` | Market, Vol, or Hedge |
| `param` | Parameter name (Near, Far, hedge, etc.) |
| `source` | Data source (BBG, BBGfc, CV, calc) |
| `ticker` | Ticker symbol or spec |
| `field` | Data field to fetch |